In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys,os,time,math,csv
import itertools
import collections

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib
import matplotlib.pyplot as plt
plt.style.use("ggplot")

import fiona
from fiona.transform import transform_geom
import pyproj
from shapely.geometry import mapping, shape
from shapely.ops import transform
from rtree import index
import rasterio
import rasterio.mask

`gdal_merge.py -o 2ft.tif -n -1 -a_nodata -1 -co COMPRESS=DEFLATE -co PREDICTOR=1 -co TILED=NO -co NUM_THREADS=ALL_CPUS /home/caleb/Dropbox/code/migration_slr/data/digital_coast/slr_2ft/*.tif`

### Mask Merged SLR Rasters

In [2]:
def process_merged_slr_mask(slr_amount):
    
    f = rasterio.open("/home/caleb/data/digital_coast/%dft.tif" % (slr_amount), "r")
    source_profile = f.profile
    data = f.read()
    f.close()
    
    dest_profile = source_profile.copy()
    dest_profile["dtype"] = rasterio.int16
    #dest_profile["compress"] = "lzw"
    #dest_profile["nodata"] = -1
    #del dest_profile["transform"]

    dest_data = data.copy()
    dest_data[dest_data>0] = 1
    dest_data[dest_data<1] = 0
    dest_data = dest_data.astype(np.int16)
    
    f = rasterio.open("/home/caleb/data/digital_coast/%dft_masked.tif" % (slr_amount), "w", **dest_profile)
    f.write(dest_data)
    f.close()

In [4]:
process_merged_slr_mask(6)

In [5]:
f = rasterio.open("/home/caleb/data/digital_coast/4ft_masked_union.tif", "r")
mask = f.read()
f.close()

for slr_amount in range(5,7):
    print(slr_amount)
    f = rasterio.open("/home/caleb/data/digital_coast/%dft_masked.tif" % (slr_amount), "r")
    profile = f.profile
    mask = np.clip(mask + f.read(), None, 1)
    f.close()

    f = rasterio.open("/home/caleb/data/digital_coast/%dft_masked_union.tif" % (slr_amount), "w", **profile)
    f.write(mask)
    f.close()

5
6


## Load block groups

In [2]:
load_time = float(time.time())

block_group_geoids = []
block_group_geoms = []
f = fiona.open("data/boundary_shapefiles/block_groups_2012/tl_2012_all_bg.shp", "r")
for s in f:
    geoid = s["properties"]["GEOID"]
    #geom = shape(s['geometry'])
    geom = s['geometry']
    
    block_group_geoids.append(geoid)
    block_group_geoms.append(geom)
f.close()

num_block_groups = len(block_group_geoids)

print("Finished loading %d block group geometries in %0.4f seconds" % (num_block_groups, time.time() - load_time))

Finished loading 216330 block group geometries in 20.6674 seconds


In [3]:
def process_block_group_intersections(slr_amount, block_group_geoms, block_group_geoids):

    area = []
    area_flooded = []

    f = rasterio.open("/home/caleb/data/digital_coast/%dft_masked_union.tif" % (slr_amount), "r")

    for i in range(num_block_groups):
        if i % 100000 == 0:
            print("\t%d/%d" % (i+1, num_block_groups))
        out_image, out_transform = rasterio.mask.mask(f, [block_group_geoms[i]], crop=True, all_touched=False)

        num_outside_shape = np.sum(out_image==-1)
        num_non_flooded = np.sum(out_image==0)
        num_flooded = np.sum(out_image==1)

        area.append(num_non_flooded + num_flooded)
        area_flooded.append(num_flooded)

        assert np.abs(out_image.size - (num_outside_shape+num_non_flooded+num_flooded)) < 1e-5
    f.close()
    
    
    f = open("data/slr_%dft_bg_intersection.csv" % (slr_amount),"w")
    f.write("GEOID,Total Area,Area Flooded,Percent Flooded\n")
    for i in range(num_block_groups):
        flooded = 0.0
        if area[i] == 0:
            #print("Block group %s" % (block_group_geoids[i]))
            flooded = 0.0
        else:
            flooded = area_flooded[i] / float(area[i])
        f.write("%s,%d,%d,%f\n" % (block_group_geoids[i], area[i], area_flooded[i], flooded))
    f.close()

    f = open("data/slr_%dft_bg_intersection.csvt" % (slr_amount),"w")
    f.write('"String","Integer","Integer","Real"')
    f.close()

In [4]:
for i in range(5, 7):
    print(i)
    process_block_group_intersections(i, block_group_geoms, block_group_geoids)

5
	1/216330
	100001/216330
	200001/216330
6
	1/216330
	100001/216330
	200001/216330
